In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import gc # Garbage Collector
import os
import sys
from sklearn.preprocessing import MinMaxScaler
path = ""

In [93]:
# Cellule à exécuter si on utilise Colab
# import kagglehub

# # à executer seulement si on utilise Colab
# # ne pas oublier que les dotenv ne sont pas sur vos machines c'est sur collab donc à vous de gérer vos variables d'environnement en local
# os.environ["KAGGLE_USERNAME"] = "eminebassoum"
# os.environ["KAGGLE_KEY"] = "347be8c875407507a6a20e6b693536d0"
# path = kagglehub.competition_download("store-sales-time-series-forecasting")
# print(f"Dataset downloaded to: {path}")
# print("Contents of the dataset directory:")
# print(os.listdir(path))

## Feature Engineering

In [94]:
# Chargement des données
# Check if files exist in 'data' directory, otherwise use kagglehub path
data_dir = 'data' if os.path.exists('data') else path

train = pd.read_csv(os.path.join(data_dir, 'train.csv'), parse_dates=['date'])
test = pd.read_csv(os.path.join(data_dir, 'test.csv'), parse_dates=['date'])
stores = pd.read_csv(os.path.join(data_dir, 'stores.csv'))
oil = pd.read_csv(os.path.join(data_dir, 'oil.csv'), parse_dates=['date'])
transactions = pd.read_csv(os.path.join(data_dir, 'transactions.csv'), parse_dates=['date'])
holidays = pd.read_csv(os.path.join(data_dir, 'holidays_events.csv'), parse_dates=['date'])

In [95]:
# Préparation initiale et fusion
train['is_train'] = 1
test['is_train'] = 0
test['sales'] = np.nan
data = pd.concat([train, test], sort=False).reset_index(drop=True)

In [96]:
# Traitement du prix du pétrole (interpolation)
oil['dcoilwtico'] = oil['dcoilwtico'].ffill().bfill()
full_dates = pd.date_range(start=data['date'].min(), end=data['date'].max())
oil = oil.set_index('date').reindex(full_dates)
oil.index.name = 'date'
oil['dcoilwtico'] = oil['dcoilwtico'].ffill().bfill()
oil = oil.reset_index()

# Fusions de base
data = data.merge(stores, on='store_nbr', how='left')
data = data.merge(oil, on='date', how='left')

In [97]:
# Attention ceci dépend du type de modèle utilisé
# Application de la transformation log(1+x) aux ventes dans le jeu d'entraînement
# data.loc[data['is_train'] == 1, 'sales'] = np.log1p(data.loc[data['is_train'] == 1, 'sales'])

# Note : attention à bien inverser cette transformation lors de la prédiction finale avec np.expm1()

In [98]:
data.head()

,id,date,store_nbr,family,sales,onpromotion,is_train,city,state,type,cluster,dcoilwtico
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,1,Quito,Pichincha,D,13,93.14
1,1,2013-01-01,1,BABY CARE,0.0,0,1,Quito,Pichincha,D,13,93.14
2,2,2013-01-01,1,BEAUTY,0.0,0,1,Quito,Pichincha,D,13,93.14
3,3,2013-01-01,1,BEVERAGES,0.0,0,1,Quito,Pichincha,D,13,93.14
4,4,2013-01-01,1,BOOKS,0.0,0,1,Quito,Pichincha,D,13,93.14


In [99]:
# Gestion des jours fériés
holidays_events = holidays[holidays['transferred'] == False]

# Séparation par portée (National, Regional, Local)
holidays_nat = holidays_events[holidays_events['locale'] == 'National'][['date', 'type']].rename(columns={'type': 'national_holiday_type'}).drop_duplicates(subset=['date'])
holidays_reg = holidays_events[holidays_events['locale'] == 'Regional'][['date', 'locale_name', 'type']].rename(columns={'locale_name': 'state', 'type': 'regional_holiday_type'})
holidays_loc = holidays_events[holidays_events['locale'] == 'Local'][['date', 'locale_name', 'type']].rename(columns={'locale_name': 'city', 'type': 'local_holiday_type'})

# Fusions des jours fériés
data = data.merge(holidays_nat, on='date', how='left')
data = data.merge(holidays_reg, on=['date', 'state'], how='left')
data = data.merge(holidays_loc, on=['date', 'city'], how='left')

# Création du flag is_holiday et nettoyage
data['is_holiday'] = (
    data['national_holiday_type'].notna() |
    data['regional_holiday_type'].notna() |
    data['local_holiday_type'].notna()
).astype(int)

data.drop(columns=['national_holiday_type', 'regional_holiday_type', 'local_holiday_type'], inplace=True)

# %%
# Vérification rapide
print(data[data['is_holiday'] == 1][['date', 'store_nbr', 'city', 'state', 'is_holiday']].head())


        date  store_nbr   city      state  is_holiday
0 2013-01-01          1  Quito  Pichincha           1
1 2013-01-01          1  Quito  Pichincha           1
2 2013-01-01          1  Quito  Pichincha           1
3 2013-01-01          1  Quito  Pichincha           1
4 2013-01-01          1  Quito  Pichincha           1


In [100]:

# Features temporelles basiques
data['day'] = data['date'].dt.day
data['month'] = data['date'].dt.month
data['year'] = data['date'].dt.year
data['dayofweek'] = data['date'].dt.dayofweek
data['weekofyear'] = data['date'].dt.isocalendar().week.astype(int)
data['is_weekend'] = (data['dayofweek'] >= 5).astype(int)
data['is_payday'] = ((data['date'].dt.is_month_end) | (data['date'].dt.day == 15)).astype(int)


In [101]:

# Intégration des transactions
data = data.merge(transactions, how="left", on=["date", "store_nbr"])

# Imputation des transactions manquantes
data.loc[data['transactions'].isnull() & (data['sales'] == 0), 'transactions'] = 0
data['transactions'] = data['transactions'].fillna(data.groupby(['store_nbr'])['transactions'].transform('mean'))

# Création des features avancées (Lags & Rolling)
print("Création des caractéristiques temporelles (ventes et transactions)...")

data.sort_values(by=['store_nbr', 'family', 'date'], inplace=True)
SHIFT_DAYS = 16

# Groupers
trans_grouper = data.groupby(['store_nbr', 'family'])['transactions']
sales_grouper = data.groupby(['store_nbr', 'family'])['sales']

# Séries décalées de base
shifted_transactions = trans_grouper.shift(SHIFT_DAYS)
base_shifted_sales = sales_grouper.shift(SHIFT_DAYS)

# --- Features Transactions ---
data[f'transactions_lag_{SHIFT_DAYS}'] = shifted_transactions
data['transactions_lag_28'] = trans_grouper.shift(28)

data['transactions_roll_mean_7'] = shifted_transactions.rolling(7).mean()
data['transactions_roll_mean_28'] = shifted_transactions.rolling(28).mean()
data['transactions_roll_std_7'] = shifted_transactions.rolling(7).std()

# --- AJOUTS: Leads, Rollings sur Exogènes, Volatilité sur Ventes ---

# 3.1. Leads de Promotion (Future Exogène) : Leads 1, 2, 3
LEADS_DAYS = [1, 2, 3]
promo_grouper = data.groupby(['store_nbr', 'family'])['onpromotion']
for lead in LEADS_DAYS:
    # Onpromotion peut varier par famille et magasin, on groupe par les deux
    data[f'onpromotion_lead_{lead}'] = promo_grouper.shift(-lead).fillna(0)


# 3.2. Moyennes Mobiles de 'onpromotion' (Future Exogène)
data['onpromotion_roll_mean_7'] = promo_grouper.rolling(window=7, min_periods=1).mean().reset_index(level=[0,1], drop=True)
data['onpromotion_roll_mean_28'] = promo_grouper.rolling(window=28, min_periods=1).mean().reset_index(level=[0,1], drop=True)


# 3.3. Moyennes Mobiles de 'oil' (Future Exogène)
# Oil est le même pour tous, on groupe seulement par date, puis on merge

# 1. Calculer les rollings sur la colonne OIL originale pour avoir une série par DATE
# Créer une série de dates complète pour l'index
full_dates = pd.date_range(start=data['date'].min(), end=data['date'].max())
oil_dates = data[['date', 'dcoilwtico']].drop_duplicates(subset=['date']).set_index('date').reindex(full_dates)
oil_dates.index.name = 'date'
oil_dates['dcoilwtico'] = oil_dates['dcoilwtico'].ffill().bfill() # (Assurez-vous que l'interpolation a été faite avant)

# Calculer les rollings sur la série temporelle journalière
oil_roll_7 = oil_dates['dcoilwtico'].rolling(window=7, min_periods=1).mean()
oil_roll_28 = oil_dates['dcoilwtico'].rolling(window=28, min_periods=1).mean()

# 2. Créer une colonne 'date' pour l'alignement sur le DataFrame 'data'
data = data.set_index('date')

# 3. Utiliser .loc pour l'alignement direct : c'est très rapide et ne duplique pas la mémoire
data['oil_roll_mean_7'] = oil_roll_7.astype(np.float32)
data['oil_roll_mean_28'] = oil_roll_28.astype(np.float32)

# 4. Remettre 'date' comme une colonne
data = data.reset_index()

# Libérer la mémoire
del oil_dates, oil_roll_7, oil_roll_28
gc.collect()

Création des caractéristiques temporelles (ventes et transactions)...


1432

In [102]:

# --- Features Ventes ---
data[f'sales_lag_{SHIFT_DAYS}'] = base_shifted_sales
data['sales_lag_23'] = base_shifted_sales.shift(7)
data['sales_lag_30'] = base_shifted_sales.shift(14)
data['sales_lag_44'] = base_shifted_sales.shift(28)
data['sales_lag_380'] = base_shifted_sales.shift(364)

data['sales_roll_7'] = base_shifted_sales.rolling(window=7, min_periods=1).mean()
data['sales_roll_14'] = base_shifted_sales.rolling(window=14, min_periods=1).mean()
data['sales_roll_28'] = base_shifted_sales.rolling(window=28, min_periods=1).mean()

# Ajouter l'écart-type de la moyenne mobile des ventes (volatilité)
# base_shifted_sales est un shift de 16 jours (pour éviter le leakage)
data['sales_roll_std_7'] = base_shifted_sales.rolling(window=7, min_periods=1).std()
data['sales_roll_std_28'] = base_shifted_sales.rolling(window=28, min_periods=1).std()

# Nettoyage final des NaN générés par les lags
data.fillna(0, inplace=True)
print("Traitement terminé.")


Traitement terminé.


In [103]:
# Ajout du Feature "Tremblement de Terre"

# Date du tremblement de terre
earthquake_date = pd.to_datetime('2016-04-16')
earthquake_window = 21 # Effet prolongé sur 21 jours

# Création d'une série temporelle simple pour le jour de l'événement
data['earthquake_impulse'] = (data['date'] == earthquake_date).astype(int)

# Création de 21 lags de cette impulsion
for i in range(1, earthquake_window + 1):
    data[f'earthquake_lag_{i}'] = data['earthquake_impulse'].shift(i)

# Remplacer les NaN (avant le début de la série) par 0
data[data.columns[data.columns.str.startswith('earthquake')]] = \
    data[data.columns[data.columns.str.startswith('earthquake')]].fillna(0.0)

# Suppression de la colonne d'impulsion de base si elle n'est pas nécessaire
data.drop(columns=['earthquake_impulse'], inplace=True)

In [104]:
# Ajout des Features de Fourier pour la Saisonnalité Annuelle

# Créer les colonnes sin/cos pour une période de 365.25 jours (Année)
# Ordre = 4 (8 colonnes, suffisant pour une bonne approximation de la saisonnalité annuelle)
def create_fourier_features(df, freq, order, prefix):
    # df['date'] est déjà en datetime64[ns]
    time = (df['date'] - df['date'].min()).dt.days
    for k in range(1, order + 1):
        df[f'{prefix}_sin_{k}'] = np.sin(2 * np.pi * k * time / freq)
        df[f'{prefix}_cos_{k}'] = np.cos(2 * np.pi * k * time / freq)
    return df

# Application des features de Fourier (Annuel)
data = create_fourier_features(data, freq=365.25, order=4, prefix='annual')

In [105]:
# --- AJOUT: Scaling des Covariables Numériques (MinMaxScaler) pour le DL ---

scaler = MinMaxScaler()

# Identification de toutes les colonnes numériques (y compris les lags/rollings/fourier)
# Exclure 'date', 'is_train', 'id', et 'sales' (car 'sales' est déjà log-transformé et doit rester séparé)
cols_to_exclude = ['date', 'is_train', 'id', 'sales'] 

# Laisser 'store_nbr' et 'cluster' pour un éventuel Embedding si vous décidez de ne pas les OHE
# Sinon, s'assurer que les colonnes OHE ne sont pas incluses (elles sont déjà 0/1)

cols_to_scale = [col for col in data.columns 
                 if data[col].dtype in [np.float64, np.float32, np.int64] 
                 and col not in cols_to_exclude 
                 and not col.startswith(('family_', 'city_', 'state_', 'type_', 'cluster_'))]

# Application du MinMax Scaling
data[cols_to_scale] = scaler.fit_transform(data[cols_to_scale])

In [106]:
# a) One-Hot Encoding pour le DL (Alternative aux Embeddings)
categorical_cols = ['family', 'city', 'state', 'type', 'cluster']
#data = pd.get_dummies(data, columns=categorical_cols, prefix=categorical_cols)

# b) Store_nbr: Le garder comme un entier pour Embeddings (si possible) ou le laisser OHE
# Si vous le voulez en OHE, il faut l'ajouter à la liste `categorical_cols` au-dessus.
# Sinon, s'assurer qu'il est de type 'int' si votre modèle DL utilise une couche d'Embedding pour lui.
# Le laisser ici en int.
data['store_nbr'] = data['store_nbr'].astype(int)

## LIGHTGBM AISTUDIO BASELINE 0.44

In [107]:
print("Début du script de baseline LightGBM...")

# ==============================================================================
# ÉTAPE 1: PRÉPARATION FINALE POUR LE MODÈLE
# ==============================================================================

print("Préparation finale des données...")

# !! CORRECTION : On ne supprime pas la colonne 'date' tout de suite !!
if 'transactions' in data.columns:
    data.drop(columns=['transactions'], inplace=True)

# Conversion des colonnes catégorielles
categorical_features = [
    "store_nbr", "family", "city", "state", "type", "cluster",
    "is_holiday", "dayofweek", "month"
]
for col in categorical_features:
    if col in data.columns:
        data[col] = data[col].astype('category')

# ==============================================================================
# ÉTAPE 2: SÉPARATION DES DONNÉES EN ENTRAÎNEMENT ET TEST
# ==============================================================================
print("Séparation des jeux d'entraînement et de test...")
train_df = data[data['is_train'] == 1].copy()
test_df = data[data['is_train'] == 0].copy()

# ==============================================================================
# ÉTAPE 3 BIS: CRÉATION D'UN JEU DE VALIDATION (MÉTHODE CORRIGÉE)
# ==============================================================================
print("Création du jeu de validation...")

# On utilise la colonne 'date' QUI EXISTE DÉJÀ DANS train_df
last_train_date = train_df['date'].max()
validation_start_date = last_train_date - pd.DateOffset(days=15)

# On crée le masque DIRECTEMENT à partir de train_df. Les longueurs correspondront parfaitement.
valid_indices = train_df[train_df['date'] >= validation_start_date].index
train_indices = train_df[train_df['date'] < validation_start_date].index

# Créer les jeux de données partiels
train_part_df = train_df.loc[train_indices]
valid_df = train_df.loc[valid_indices]

print(f"Jeu d'entraînement partiel : {train_part_df.shape[0]} lignes")
print(f"Jeu de validation : {valid_df.shape[0]} lignes")

# ==============================================================================
# ÉTAPE 4: DÉFINITION DES FEATURES (X) ET DE LA CIBLE (y)
# ==============================================================================

# La cible est la colonne 'sales'
y_train_part = np.log1p(train_part_df['sales'])
y_valid = np.log1p(valid_df['sales'])

# Les features sont toutes les autres colonnes utiles
# !! CORRECTION : On supprime 'date' et les autres colonnes inutiles ICI !!
features = [col for col in train_df.columns if col not in ['id', 'sales', 'is_train', 'date']]

X_train_part = train_part_df[features]
X_valid = valid_df[features]
X_test = test_df[features] # On utilise la même liste de features pour le test

# Garder les IDs pour la soumission
test_ids = test_df['id'].copy()

# Libérer de la mémoire
del train_df, test_df, data, train_part_df, valid_df
gc.collect()

# ==============================================================================
# ÉTAPE 5: ENTRAÎNEMENT DU MODÈLE LIGHTGBM (inchangé)
# ==============================================================================
print("\nEntraînement du modèle LightGBM avec suivi de la progression...")

lgb_params = {
    'objective': 'regression_l1', 'metric': 'rmse', 'n_estimators': 2000,
    'learning_rate': 0.02, 'feature_fraction': 0.8, 'bagging_fraction': 0.8,
    'bagging_freq': 1, 'lambda_l1': 0.1, 'lambda_l2': 0.1, 'num_leaves': 31,
    'verbose': -1, 'n_jobs': -1, 'seed': 42, 'boosting_type': 'gbdt',
}

model = lgb.LGBMRegressor(**lgb_params)

model.fit(X_train_part, y_train_part,
          eval_set=[(X_valid, y_valid)],
          eval_metric='rmse',
          callbacks=[
              lgb.log_evaluation(period=100),
              lgb.early_stopping(stopping_rounds=100)
          ],
          categorical_feature=[col for col in categorical_features if col in features])

# ==============================================================================
# ÉTAPE 6: PRÉDICTION ET CRÉATION DU FICHIER DE SOUMISSION (inchangé)
# ==============================================================================
print("\nGénération des prédictions sur le jeu de test...")

best_iteration = model.best_iteration_ if model.best_iteration_ else lgb_params['n_estimators']
print(f"Meilleure itération trouvée : {best_iteration}")

predictions_log = model.predict(X_test, num_iteration=best_iteration)
predictions = np.expm1(predictions_log)
predictions[predictions < 0] = 0

submission_df = pd.DataFrame({'id': test_ids, 'sales': predictions})
submission_df.to_csv('submission_baseline_lgbm_v2.csv', index=False)

print("\nFichier 'submission_baseline_lgbm_v2.csv' créé avec succès !")
print(submission_df.head())

Début du script de baseline LightGBM...
Préparation finale des données...
Séparation des jeux d'entraînement et de test...
Création du jeu de validation...
Jeu d'entraînement partiel : 2972640 lignes
Jeu de validation : 28512 lignes

Entraînement du modèle LightGBM avec suivi de la progression...
Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 0.817864
[200]	valid_0's rmse: 0.607554
[300]	valid_0's rmse: 0.585502
[400]	valid_0's rmse: 0.576006
[500]	valid_0's rmse: 0.570561
[600]	valid_0's rmse: 0.565272
[700]	valid_0's rmse: 0.560235
[800]	valid_0's rmse: 0.558654
[900]	valid_0's rmse: 0.557284
[1000]	valid_0's rmse: 0.5545
[1100]	valid_0's rmse: 0.552796
[1200]	valid_0's rmse: 0.552145
[1300]	valid_0's rmse: 0.551086
[1400]	valid_0's rmse: 0.549513
[1500]	valid_0's rmse: 0.548373
[1600]	valid_0's rmse: 0.54536
[1700]	valid_0's rmse: 0.541268
[1800]	valid_0's rmse: 0.536977
[1900]	valid_0's rmse: 0.53192
[2000]	valid_0's rmse: 0.523008
Did not meet 